In [1]:
import re

class Morph():
    def __init__(self, doc_list):
        self.surface = doc_list[0]
        self.base = doc_list[7]
        self.pos = doc_list[1]
        self.pos1= doc_list[2]

    def __str__(self):
        return "{" + f"surface: {self.surface}, base: {self.base}, pos: {self.pos}, pos1: {self.pos1}" + "}"

class Chunk():
    def __init__(self,morph_list,dst,srcs):
        self.morph_list = morph_list
        self.dst = int(dst)
        self.srcs = srcs

    def __str__(self):
        morphs_str = ', '.join(str(morph) for morph in self.morph_list) # Morphオブジェクトのリストを文字列に変換
        return f"Morphs: [{morphs_str}], Dst: {self.dst}, Srcs: {self.srcs}"

# 文節番号と係り先番号の取得
def find_feature_number(text):
    pattern = r"^\* \d+ ([^\d]*\d+)D"
    match = re.search(pattern, text)
    if match:
        dst_number = match.group(1)  # 文節番号  # 係先番号
        return dst_number
    else:
        return None

In [2]:
with open(file="ai.ja.txt.parsed", mode="r", encoding="utf-8")as f:
    docs = f.readlines()

Chunk_list = []
chunk_flag = False
# srcの追加処理は後で行う
for doc in docs[5:]:
    if doc == "EOS\n":
        chunk = Chunk(morph_list,dst_num,[])
        Chunk_list.append(chunk)
        break
    # *から始まる行とEOSの行は飛ばす
    if doc[0]=="*":
        if chunk_flag:
            chunk = Chunk(morph_list,dst_num,[])
            Chunk_list.append(chunk)
        dst_num = find_feature_number(doc)
        morph_list = []
        chunk_flag = True
    else:
        doc = doc.replace("\t",",").replace("\n", "")
        doc_list = doc.split(",")
        doc_morph = Morph(doc_list)
        morph_list.append(doc_morph)


# Srcsを埋めていく
for i in range(len(Chunk_list)):
    Chunk_list[Chunk_list[i].dst].srcs.append(i)

In [3]:
# 41を再利用していく

# morphのsurfaceのみを抽出して，文章を再構築する
def get_sentence(chunk: Chunk):
    remove_list = ["（", "）", "、", "。", "「", "」", "『", "』", "〈", "〉"]
    morph_list = chunk.morph_list
    sentence_str = ""
    for morph in morph_list:
        if morph.surface in remove_list:
            continue
        sentence_str += morph.surface
    return sentence_str

# typeはnoun or verb　のみ
def has_type_in_sentence(chunk: Chunk, pos) -> bool:
    morph_list = chunk.morph_list
    if pos == "noun":
        for morph in morph_list:
            if morph.pos == "名詞":
                return True
        return False
    elif pos == "verb":
        for morph in morph_list:
            if morph.pos == "動詞":
                return True
        return False
    else:
        print("Invalid type. Put noun or verb into pos.")

for chunk in Chunk_list:
    src_morph = chunk
    dst_morph = Chunk_list[chunk.dst]
    has_noun_in_src_sentence = has_type_in_sentence(src_morph, "noun")
    has_verb_in_dst_sentence = has_type_in_sentence(dst_morph, "verb")
    if  has_noun_in_src_sentence and has_verb_in_dst_sentence:
        result = get_sentence(src_morph) + "\t" + get_sentence(dst_morph)
        print(result)

道具を	用いて
知能を	研究する
一分野を	指す
知的行動を	代わって
人間に	代わって
コンピューターに	行わせる
研究分野とも	される
